In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Dropout, Conv2D, BatchNormalization, Flatten, Dense, MaxPool2D, Input, Activation, Add, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, Adamax, RMSprop
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import json

In [2]:
fldr = "jangedoo/utkface-new/versions/1/UTKFace"

In [3]:
files = os.listdir(fldr)

In [4]:
ages = []
genders = []
images = []

In [5]:
for f in files:
    age = int(f.split('_')[0])
    ages.append(age)
    gender = int(f.split('_')[1])
    genders.append(gender)
    image = cv2.imread(fldr + '/' + f)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (48, 48))
    images.append(image)

In [6]:
images_f  = np.array(images)
ages_f = np.array(ages)
genders_f = np.array(genders)

In [7]:
# Defining labels for the model

labels = []
i = 0
while i < len(ages_f):
    label = []
    label.append(ages_f[i])
    label.append(genders_f[i])
    labels.append(label)
    i += 1

In [8]:
labels_f = np.array(labels)

In [9]:
# Creating another list of images with the pixel values scaled between 0 and 1

images_f_2 = images_f / 255.0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(images_f_2, labels_f, test_size=0.25, random_state=42)

In [11]:
y_train_2=[y_train[:,1],y_train[:,0]]
y_test_2=[y_test[:,1],y_test[:,0]]

In [12]:
# using Tensorflow

def Convolution(input_tensor, filters):
    x = Conv2D(filters=filters, kernel_size=(3,3), padding="same", strides=(1,1), kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x = Activation('relu')(x)
    return x

In [13]:
def model(input_shape):
    inputs = Input((input_shape))
    conv_1 = Convolution(inputs, 32)
    maxp_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2 = Convolution(maxp_1, 64)
    maxp_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3 = Convolution(maxp_2, 128)
    maxp_3 = MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4 = Convolution(maxp_3, 256)
    maxp_4 = MaxPooling2D(pool_size=(2,2))(conv_4)
    flatten = Flatten()(maxp_4)
    dense_1 = Dense(64, activation='relu')(flatten)
    dense_2 = Dense(64, activation='relu')(flatten)
    drop_1 = Dropout(0.2)(dense_1)
    drop_2 = Dropout(0.2)(dense_2)
    output_1 = Dense(1, activation='sigmoid', name='gender_out')(drop_1)
    output_2 = Dense(1, activation='relu', name='age_out')(drop_2)
    model = Model(inputs=inputs, outputs=[output_1, output_2])
    model.compile(optimizer='adam', loss=['binary_crossentropy', 'mae'], metrics=['accuracy', 'accuracy'])
    return model

In [14]:
model = model((48, 48, 3))

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 48, 48,    │          0 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 48, 48,    │          0 │ dropout[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 24, 24,    │          0 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 24, 24,    │          0 │ dropout_1[0][0]   │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 12, 12,    │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 12, 12,    │          0 │ dropout_2[0][0]   │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 6, 6, 256) │    295,168 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 6, 6, 256) │          0 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 6, 6, 256) │          0 │ dropout_3[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 3, 3, 256) │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 683,586 (2.61 MB)

 Trainable params: 683,586 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
fle_s = 'Age_Gender_Detection.h5'
checkpoint = ModelCheckpoint(fle_s, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
early_stop = EarlyStopping(monitor='val_loss', patience=75, restore_best_weights=True)
callback_list = [checkpoint, early_stop]

In [17]:
history = model.fit(X_train, y_train_2, batch_size=64, validation_data=(X_test,y_test_2), epochs=250, callbacks=callback_list)

Epoch 1/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - age_out_accuracy: 0.0463 - age_out_loss: 17.3237 - gender_out_accuracy: 0.5436 - gender_out_loss: 0.6924 - loss: 18.2306
Epoch 1: val_loss improved from inf to 16.27159, saving model to Age_Gender_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 27s 94ms/step - age_out_accuracy: 0.0463 - age_out_loss: 17.3183 - gender_out_accuracy: 0.5437 - gender_out_loss: 0.6924 - loss: 18.2252 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 15.4347 - val_gender_out_accuracy: 0.6194 - val_gender_out_loss: 0.6408 - val_loss: 16.2716
Epoch 2/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - age_out_accuracy: 0.0449 - age_out_loss: 12.1886 - gender_out_accuracy: 0.6908 - gender_out_loss: 0.5902 - loss: 13.0045
Epoch 2: val_loss improved from 16.27159 to 12.39520, saving model to Age_Gender_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - age_out_accuracy: 0.0449 - age_out_loss: 12.1851 - gender_out_accuracy: 0.6909 - gender_out_loss: 0.5901 - loss: 13.0010 - val_age_out_accuracy: 0.0472 - val_age_out_loss: 11.5724 - val_gender_out_accuracy: 0.7550 - val_gender_out_loss: 0.5169 - val_loss: 12.3952
Epoch 3/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - age_out_accuracy: 0.0415 - age_out_loss: 9.6666 - gender_out_accuracy: 0.7517 - gender_out_loss: 0.5055 - loss: 10.4775
Epoch 3: val_loss improved from 12.39520 to 10.41539, saving model to Age_Gender_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 27s 97ms/step - age_out_accuracy: 0.0416 - age_out_loss: 9.6659 - gender_out_accuracy: 0.7518 - gender_out_loss: 0.5055 - loss: 10.4767 - val_age_out_accuracy: 0.0472 - val_age_out_loss: 9.6337 - val_gender_out_accuracy: 0.8055 - val_gender_out_loss: 0.4444 - val_loss: 10.4154
Epoch 4/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - age_out_accuracy: 0.0461 - age_out_loss: 8.8491 - gender_out_accuracy: 0.7827 - gender_out_loss: 0.4584 - loss: 9.6425
Epoch 4: val_loss did not improve from 10.41539
278/278 ━━━━━━━━━━━━━━━━━━━━ 27s 97ms/step - age_out_accuracy: 0.0461 - age_out_loss: 8.8489 - gender_out_accuracy: 0.7828 - gender_out_loss: 0.4583 - loss: 9.6423 - val_age_out_accuracy: 0.0471 - val_age_out_loss: 12.4467 - val_gender_out_accuracy: 0.8058 - val_gender_out_loss: 0.4251 - val_loss: 13.2342
Epoch 5/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - age_out_accuracy: 0.0467 - age_out_loss: 8.4719 - gender_out_accuracy: 0.7941 - gender_out_loss: 0.4354 - l

278/278 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - age_out_accuracy: 0.0454 - age_out_loss: 7.9503 - gender_out_accuracy: 0.8131 - gender_out_loss: 0.3986 - loss: 8.7333 - val_age_out_accuracy: 0.0471 - val_age_out_loss: 8.5579 - val_gender_out_accuracy: 0.8281 - val_gender_out_loss: 0.3835 - val_loss: 9.3479
Epoch 7/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - age_out_accuracy: 0.0443 - age_out_loss: 7.8688 - gender_out_accuracy: 0.8212 - gender_out_loss: 0.3738 - loss: 8.6510
Epoch 7: val_loss did not improve from 9.34788
278/278 ━━━━━━━━━━━━━━━━━━━━ 33s 118ms/step - age_out_accuracy: 0.0443 - age_out_loss: 7.8689 - gender_out_accuracy: 0.8212 - gender_out_loss: 0.3738 - loss: 8.6511 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 9.2997 - val_gender_out_accuracy: 0.8397 - val_gender_out_loss: 0.3590 - val_loss: 10.0855
Epoch 8/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - age_out_accuracy: 0.0489 - age_out_loss: 7.4595 - gender_out_accuracy: 0.8234 - gender_out_loss: 0.3735 - l

278/278 ━━━━━━━━━━━━━━━━━━━━ 31s 112ms/step - age_out_accuracy: 0.0451 - age_out_loss: 6.9383 - gender_out_accuracy: 0.8475 - gender_out_loss: 0.3325 - loss: 7.7448 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.8626 - val_gender_out_accuracy: 0.8681 - val_gender_out_loss: 0.2957 - val_loss: 8.6500
Epoch 12/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - age_out_accuracy: 0.0451 - age_out_loss: 6.8053 - gender_out_accuracy: 0.8630 - gender_out_loss: 0.3031 - loss: 7.5994
Epoch 12: val_loss did not improve from 8.65001
278/278 ━━━━━━━━━━━━━━━━━━━━ 31s 110ms/step - age_out_accuracy: 0.0451 - age_out_loss: 6.8058 - gender_out_accuracy: 0.8630 - gender_out_loss: 0.3031 - loss: 7.5999 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 9.5693 - val_gender_out_accuracy: 0.8699 - val_gender_out_loss: 0.3045 - val_loss: 10.3815
Epoch 13/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - age_out_accuracy: 0.0479 - age_out_loss: 6.5689 - gender_out_accuracy: 0.8660 - gender_out_loss: 0.3135 

278/278 ━━━━━━━━━━━━━━━━━━━━ 29s 106ms/step - age_out_accuracy: 0.0462 - age_out_loss: 6.5984 - gender_out_accuracy: 0.8678 - gender_out_loss: 0.2932 - loss: 7.4263 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.1522 - val_gender_out_accuracy: 0.8888 - val_gender_out_loss: 0.2587 - val_loss: 7.9581
Epoch 16/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - age_out_accuracy: 0.0488 - age_out_loss: 6.1958 - gender_out_accuracy: 0.8766 - gender_out_loss: 0.2770 - loss: 7.0227
Epoch 16: val_loss did not improve from 7.95812
278/278 ━━━━━━━━━━━━━━━━━━━━ 29s 105ms/step - age_out_accuracy: 0.0488 - age_out_loss: 6.1965 - gender_out_accuracy: 0.8766 - gender_out_loss: 0.2770 - loss: 7.0234 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.4267 - val_gender_out_accuracy: 0.8821 - val_gender_out_loss: 0.2647 - val_loss: 8.2548
Epoch 17/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - age_out_accuracy: 0.0463 - age_out_loss: 6.1110 - gender_out_accuracy: 0.8752 - gender_out_loss: 0.2791 - l

278/278 ━━━━━━━━━━━━━━━━━━━━ 31s 113ms/step - age_out_accuracy: 0.0455 - age_out_loss: 5.6642 - gender_out_accuracy: 0.8970 - gender_out_loss: 0.2421 - loss: 6.6012 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.9789 - val_gender_out_accuracy: 0.8951 - val_gender_out_loss: 0.2490 - val_loss: 7.9322
Epoch 27/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - age_out_accuracy: 0.0482 - age_out_loss: 5.5278 - gender_out_accuracy: 0.8991 - gender_out_loss: 0.2398 - loss: 6.4740
Epoch 27: val_loss did not improve from 7.93215
278/278 ━━━━━━━━━━━━━━━━━━━━ 30s 108ms/step - age_out_accuracy: 0.0482 - age_out_loss: 5.5279 - gender_out_accuracy: 0.8991 - gender_out_loss: 0.2398 - loss: 6.4741 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.3820 - val_gender_out_accuracy: 0.8856 - val_gender_out_loss: 0.2651 - val_loss: 8.3688
Epoch 28/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - age_out_accuracy: 0.0452 - age_out_loss: 5.4825 - gender_out_accuracy: 0.9000 - gender_out_loss: 0.2328 - 

278/278 ━━━━━━━━━━━━━━━━━━━━ 33s 118ms/step - age_out_accuracy: 0.0463 - age_out_loss: 5.3016 - gender_out_accuracy: 0.9076 - gender_out_loss: 0.2265 - loss: 6.2727 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.7747 - val_gender_out_accuracy: 0.8768 - val_gender_out_loss: 0.2677 - val_loss: 7.7962
Epoch 31/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - age_out_accuracy: 0.0463 - age_out_loss: 5.2182 - gender_out_accuracy: 0.8981 - gender_out_loss: 0.2366 - loss: 6.2112
Epoch 31: val_loss did not improve from 7.79618
278/278 ━━━━━━━━━━━━━━━━━━━━ 31s 111ms/step - age_out_accuracy: 0.0463 - age_out_loss: 5.2186 - gender_out_accuracy: 0.8981 - gender_out_loss: 0.2366 - loss: 6.2116 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 8.9751 - val_gender_out_accuracy: 0.8741 - val_gender_out_loss: 0.2579 - val_loss: 10.0013
Epoch 32/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - age_out_accuracy: 0.0486 - age_out_loss: 5.3668 - gender_out_accuracy: 0.9035 - gender_out_loss: 0.2277 

278/278 ━━━━━━━━━━━━━━━━━━━━ 32s 114ms/step - age_out_accuracy: 0.0466 - age_out_loss: 5.1735 - gender_out_accuracy: 0.9067 - gender_out_loss: 0.2203 - loss: 6.1960 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.5565 - val_gender_out_accuracy: 0.8940 - val_gender_out_loss: 0.2425 - val_loss: 7.6079
Epoch 36/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - age_out_accuracy: 0.0488 - age_out_loss: 5.2116 - gender_out_accuracy: 0.9088 - gender_out_loss: 0.2200 - loss: 6.2423
Epoch 36: val_loss did not improve from 7.60792
278/278 ━━━━━━━━━━━━━━━━━━━━ 29s 105ms/step - age_out_accuracy: 0.0488 - age_out_loss: 5.2117 - gender_out_accuracy: 0.9087 - gender_out_loss: 0.2200 - loss: 6.2425 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.8429 - val_gender_out_accuracy: 0.8934 - val_gender_out_loss: 0.2407 - val_loss: 7.9070
Epoch 37/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - age_out_accuracy: 0.0487 - age_out_loss: 5.1519 - gender_out_accuracy: 0.9032 - gender_out_loss: 0.2242 - 

In [18]:
import json

# Save history as JSON
with open("training_history.json", "w") as f:
    json.dump(history.history, f)